In [7]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN


In [8]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker


In [9]:
dataset = pd.read_csv(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/AgNN/AgBO_loss.csv')
X = dataset.iloc[:,4:10]
X




,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
0,13.101322,32.995595,5.603524,4.528634,227.0,0.579592
1,13.101322,32.995595,5.603524,4.528634,227.0,0.581558
2,13.101322,32.995595,5.603524,4.528634,227.0,0.582454
3,13.101322,32.995595,5.603524,4.528634,227.0,0.582043
4,13.101322,32.995595,5.603524,4.528634,227.0,0.592302
...,...,...,...,...,...,...
3290,36.501053,14.000000,10.501053,4.501053,950.0,0.264635
3291,36.501053,14.000000,10.501053,4.501053,950.0,0.265406
3292,36.501053,14.000000,10.501053,4.501053,950.0,0.263623
3293,36.501053,14.000000,10.501053,4.501053,950.0,0.252167


In [10]:
X['Qtot(uL/min)'] = X['Qtot(uL/min)'] / 30
X

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
0,13.101322,32.995595,5.603524,4.528634,7.566667,0.579592
1,13.101322,32.995595,5.603524,4.528634,7.566667,0.581558
2,13.101322,32.995595,5.603524,4.528634,7.566667,0.582454
3,13.101322,32.995595,5.603524,4.528634,7.566667,0.582043
4,13.101322,32.995595,5.603524,4.528634,7.566667,0.592302
...,...,...,...,...,...,...
3290,36.501053,14.000000,10.501053,4.501053,31.666667,0.264635
3291,36.501053,14.000000,10.501053,4.501053,31.666667,0.265406
3292,36.501053,14.000000,10.501053,4.501053,31.666667,0.263623
3293,36.501053,14.000000,10.501053,4.501053,31.666667,0.252167


In [14]:
features = ['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)']
X_feature = X[['QAgNO3(%)', 'Qpva(%)', 'Qtsc(%)', 'Qseed(%)', 'Qtot(uL/min)']].values

X_loss = np.array(X['loss'].values)

In [15]:
bounds = [{'name':'QAgNO3','type':'continuous','domain':(0.5,80)},
          {'name':'Qpva','type':'continuous','domain':(10,40)},
          {'name':'QNaOH','type':'continuous','domain':(0.5,80)},
          {'name':'Qhydra','type':'continuous','domain':(0.5,80)},
          {'name':'Qoil','type':'continuous','domain':(0,40)}
         ]

constraints_baseline = [{'name': 'constr_1', 'constraint': 'x[:,0] + x[:,1] + x[:,2] + x[:,3] - 90'}
                       ]

jitter = 0.01

# aaa = GPyOpt.util.general.normalize(np.array([[i] for i in X_loss]), normalization_type='maxmin')

BO_teacher = GPyOpt.methods.BayesianOptimization(f=None,  
                                                 domain = bounds,
                                                 constraints = constraints_baseline,
                                                 acquisition_type = 'EI',              
                                                 normalize_Y = True,
                                                 X = np.array(X_feature) ,
                                                 Y = np.array([[i] for i in X_loss]),
                                                 evaluator_type = 'local_penalization',
                                                 batch_size = 1,
                                                 acquisition_jitter = jitter)

# we don't care yet, this line here is to just fit the GP
_ = BO_teacher.suggest_next_locations()

# Generate grid with 1% resolution
# grid_linspace = np.linspace(0, 1, 101)
# grid_array = np.array([999, 999, 999])

# for i in grid_linspace:
#     for j in grid_linspace:
#         for k in grid_linspace:
#             x = np.array([i, j, k])
#             if i + j + k == 1:
#                 grid_array = np.vstack((grid_array, x))
# grid_array = grid_array[1:]

# denormalazing
input_mean = np.mean(X_loss)
input_std = np.std(X_loss)

# grid_pred_mean = BO_teacher.model.predict(grid_array)[0][:,0] * input_std + input_mean
# grid_pred_mean_std = BO_teacher.model.predict(grid_array)[1][:,0]* input_std

# plot_tri(grid_array[:,0],grid_array[:,1],grid_array[:,2], grid_pred_mean, 'GP model', False)
# plot_tri(X_Cs, X_MA, X_FA, X_loss, 'raw experimental', True)


In [28]:
GP_ground_loss = BO_teacher.model.predict(np.array(X_feature))[0][:,0] * input_std + input_mean
X_GP_ground = X.copy()
X_GP_ground['loss'] = GP_ground_loss
X_GP_ground.sort_values(['loss']).head(100)

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
3032,32.501176,16.000000,6.501176,4.501176,28.333333,0.150756
3018,32.501176,16.000000,6.501176,4.501176,28.333333,0.150756
3019,32.501176,16.000000,6.501176,4.501176,28.333333,0.150756
3020,32.501176,16.000000,6.501176,4.501176,28.333333,0.150756
3021,32.501176,16.000000,6.501176,4.501176,28.333333,0.150756
...,...,...,...,...,...,...
2247,33.360430,20.100503,5.108878,5.399547,27.064000,0.177240
2246,33.360430,20.100503,5.108878,5.399547,27.064000,0.177240
2258,33.360430,20.100503,5.108878,5.399547,27.064000,0.177240
2259,33.360430,20.100503,5.108878,5.399547,27.064000,0.177240


In [16]:
# batch and rounds for BO
batch = 1
rounds = 100
n_eval = batch * rounds
jitter = 0.01

seed_list = [74,660,861,1526,2,151,19,7,33,4021,66,42424,671,25,95711,850,71,2148,26219,4]
# 20

In [17]:
import pyDOE
import random

X_BO_initRS_EI_collection = []
run_BO_initRS_EI_collection = []
loss_BO_initRS_EI_collection = []

for s in seed_list:
    
    # initial_sampling = constrained random sampling = cRS
    # optimization = GpyOpt

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_EI = []
    run_BO_initRS_EI = []
    loss_BO_initRS_EI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_EI.append(initial_X[j])
                run_BO_initRS_EI.append(i)
                loss_BO_initRS_EI.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'EI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_EI),
                                            Y = np.array([[i] for i in loss_BO_initRS_EI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
#             suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_EI.append(suggested_next_X[j])
                run_BO_initRS_EI.append(i)
                loss_BO_initRS_EI.append(suggested_next_loss[j])

#             min_round = np.min(suggested_next_loss)
#             min_index = np.where(suggested_next_loss == min_round)
#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))
    X_BO_initRS_EI_collection.append(X_BO_initRS_EI)
    run_BO_initRS_EI_collection.append(run_BO_initRS_EI)
    loss_BO_initRS_EI_collection.append(loss_BO_initRS_EI)
    print('Finished e')
    
master = np.array([X_BO_initRS_EI_collection, run_BO_initRS_EI_collection, loss_BO_initRS_EI_collection])
np.save('GP_BO_EI_master', master)
    
    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')


Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [18]:
# random sampling

X_RS_collection = []
loss_RS_collection = []

for s in seed_list:

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_rs = sampler.get_samples(n_eval)
    loss_RS = BO_teacher.model.predict(X_rs)[0][:,0] * input_std + input_mean
#     loss_RS = bst.predict(xgb.DMatrix(X_rs))
    

    # plot_tri(X_rs[:,0], X_rs[:,1], X_rs[:,2], loss_RS, 'random sampling', True)
    X_RS_collection.append(X_rs)
    loss_RS_collection.append(loss_RS)
    print('Finished e')
    
master = np.array([X_RS_collection, 0, loss_RS_collection])
np.save('GP_RS_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [19]:
X_BO_initRS_MPI_collection = []
run_BO_initRS_MPI_collection = []
loss_BO_initRS_MPI_collection = []

for s in seed_list:
    random.seed(s)

    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_MPI = []
    run_BO_initRS_MPI = []
    loss_BO_initRS_MPI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(initial_X[j])
                run_BO_initRS_MPI.append(i)
                loss_BO_initRS_MPI.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'MPI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_MPI),
                                            Y = np.array([[i] for i in loss_BO_initRS_MPI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
#             suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(suggested_next_X[j])
                run_BO_initRS_MPI.append(i)
                loss_BO_initRS_MPI.append(suggested_next_loss[j])

            min_round = np.min(suggested_next_loss)
            min_index = np.where(suggested_next_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_MPI_collection.append(X_BO_initRS_MPI)
    run_BO_initRS_MPI_collection.append(run_BO_initRS_MPI)
    loss_BO_initRS_MPI_collection.append(loss_BO_initRS_MPI)
    print('Finished e')
    
master = np.array([X_BO_initRS_MPI_collection, run_BO_initRS_MPI_collection, loss_BO_initRS_MPI_collection])
np.save('GP_BO_MPI_master', master)


Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [20]:
X_BO_initRS_LCB_collection = []
run_BO_initRS_LCB_collection = []
loss_BO_initRS_LCB_collection = []

for s in seed_list:
    random.seed(s)
    
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_LCB = []
    run_BO_initRS_LCB = []
    loss_BO_initRS_LCB = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(initial_X[j])
                run_BO_initRS_LCB.append(i)
                loss_BO_initRS_LCB.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'LCB',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_LCB),
                                            Y = np.array([[i] for i in loss_BO_initRS_LCB]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
#             suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(suggested_next_X[j])
                run_BO_initRS_LCB.append(i)
                loss_BO_initRS_LCB.append(suggested_next_loss[j])

            min_round = np.min(suggested_next_loss)
            min_index = np.where(suggested_next_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_LCB_collection.append(X_BO_initRS_LCB)
    run_BO_initRS_LCB_collection.append(run_BO_initRS_LCB)
    loss_BO_initRS_LCB_collection.append(loss_BO_initRS_LCB)
    print('Finished e')
    
master = np.array([X_BO_initRS_LCB_collection, run_BO_initRS_LCB_collection, loss_BO_initRS_LCB_collection])
np.save('GP_BO_LCB_master', master)

The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ign

In [21]:
# # grid search + random sampling
# from random import sample

# grid_linspace = np.linspace(0, 1, 50)
# grid_array = np.array([999, 999, 999])

# for i in grid_linspace:
#     for j in grid_linspace:
#         for k in grid_linspace:
#             x = np.array([i, j, k])
#             if i + j + k == 1:
#                 grid_array = np.vstack((grid_array, x))
# grid_array = grid_array[1:]

# # grid_pred_mean = BO_teacher.model.predict(grid_array)[0][:,0] * input_std + input_mean
# # grid_pred_mean_std = BO_teacher.model.predict(grid_array)[1][:,0]* input_std

# grid_pred = bst.predict(xgb.DMatrix(grid_array))

# X_grid_collection = []
# loss_grid_collection = []

# for s in seed_list:
#     random.seed(s)
#     list_index = sample(list(np.arange(len(grid_pred_mean))),n_eval)
#     X_grid = []
#     loss_grid = []
#     for i in list_index:
#         X_grid.append(grid_array[i])
#         loss_grid.append(grid_pred_mean[i])
#     X_grid_collection.append(X_grid)
#     loss_grid_collection.append(loss_grid)
#     print('Finished e')
    
# master = np.array([X_grid_collection, 0, loss_grid_collection])
# np.save('Grid_master', master)
# hide_toggle('Grid')


In [22]:
import xgboost as xgb
# https://xgboost.readthedocs.io/en/latest/parameter.html
num_round = 3000
dtrain = xgb.DMatrix(np.array(X_feature), label=X_loss)

# tt = []
# for depth_ in [2,3,4,5,6,7,8,9,10]:
#     for eta_ in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
#         param = {'max_depth': depth_, 'eta': eta_, 'objective': 'reg:squarederror'}
#         bst = xgb.train(param, dtrain, num_round)
#         pred = bst.predict(xgb.DMatrix(X_feature.to_numpy()))
#         mae = np.sum((pred - X_loss)**2) / len(X_loss)
#         tt.append([depth_, eta_, mae])

# opt_XGboost = pd.DataFrame(tt, columns=['depth_', 'eta_', 'mae'])

param = {'max_depth': 6, 'eta': 0.3, 'objective': 'reg:squarederror'}
bst = xgb.train(param, dtrain, num_round)
# plot_tri(grid_array[:,0],grid_array[:,1],grid_array[:,2], bst.predict(xgb.DMatrix(grid_array)), 'XGboost model', False)
# plot_tri(X_Cs, X_MA, X_FA, X_loss, 'raw experimental', True)

# Note
# 1.
# pred = bst.predict(xgb.DMatrix(X_feature.to_numpy()))
# mae = np.sum((pred - X_loss)**2) / len(X_loss)
# <
# pred = BO_teacher.model.predict(X_feature.to_numpy())[0][:,0] * input_std + input_mean
# mae = np.sum((pred - X_loss)**2) / len(X_loss)
# 2.
# XGboost input not normalized. not necessary

In [31]:
XGB_ground_loss = bst.predict(xgb.DMatrix(np.array(X_feature)))
X_XGB_ground = X.copy()
X_XGB_ground['loss'] = XGB_ground_loss
X_XGB_ground.sort_values(['loss']).head(25)

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
3023,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3017,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3018,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3019,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3020,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3021,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3022,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3029,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3024,32.501176,16.0,6.501176,4.501176,28.333333,0.148798
3030,32.501176,16.0,6.501176,4.501176,28.333333,0.148798


In [23]:
X_BO_initRS_EI_collection = []
run_BO_initRS_EI_collection = []
loss_BO_initRS_EI_collection = []

for s in seed_list:
    
    # initial_sampling = constrained random sampling = cRS
    # optimization = GpyOpt

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_EI = []
    run_BO_initRS_EI = []
    loss_BO_initRS_EI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_EI.append(initial_X[j])
                run_BO_initRS_EI.append(i)
                loss_BO_initRS_EI.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'EI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_EI),
                                            Y = np.array([[i] for i in loss_BO_initRS_EI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

#             suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_EI.append(suggested_next_X[j])
                run_BO_initRS_EI.append(i)
                loss_BO_initRS_EI.append(suggested_next_loss[j])

#             min_round = np.min(suggested_next_loss)
#             min_index = np.where(suggested_next_loss == min_round)
#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))
    X_BO_initRS_EI_collection.append(X_BO_initRS_EI)
    run_BO_initRS_EI_collection.append(run_BO_initRS_EI)
    loss_BO_initRS_EI_collection.append(loss_BO_initRS_EI)
    print('Finished e')
    
master = np.array([X_BO_initRS_EI_collection, run_BO_initRS_EI_collection, loss_BO_initRS_EI_collection])
np.save('XGB_BO_EI_master', master)
    
    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')




Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [24]:
X_RS_collection = []
loss_RS_collection = []

for s in seed_list:

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_rs = sampler.get_samples(n_eval)
#     loss_RS = BO_teacher.model.predict(X_rs)[0][:,0] * input_std + input_mean
    loss_RS = bst.predict(xgb.DMatrix(X_rs))
    

    # plot_tri(X_rs[:,0], X_rs[:,1], X_rs[:,2], loss_RS, 'random sampling', True)
    X_RS_collection.append(X_rs)
    loss_RS_collection.append(loss_RS)
    print('Finished e')
    
master = np.array([X_RS_collection, 0, loss_RS_collection])
np.save('XGB_RS_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [25]:
X_BO_initRS_MPI_collection = []
run_BO_initRS_MPI_collection = []
loss_BO_initRS_MPI_collection = []

for s in seed_list:
    random.seed(s)

    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_MPI = []
    run_BO_initRS_MPI = []
    loss_BO_initRS_MPI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(initial_X[j])
                run_BO_initRS_MPI.append(i)
                loss_BO_initRS_MPI.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'MPI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_MPI),
                                            Y = np.array([[i] for i in loss_BO_initRS_MPI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

#             suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(suggested_next_X[j])
                run_BO_initRS_MPI.append(i)
                loss_BO_initRS_MPI.append(suggested_next_loss[j])

            min_round = np.min(suggested_next_loss)
            min_index = np.where(suggested_next_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_MPI_collection.append(X_BO_initRS_MPI)
    run_BO_initRS_MPI_collection.append(run_BO_initRS_MPI)
    loss_BO_initRS_MPI_collection.append(loss_BO_initRS_MPI)
    print('Finished e')
    
master = np.array([X_BO_initRS_MPI_collection, run_BO_initRS_MPI_collection, loss_BO_initRS_MPI_collection])
np.save('XGB_BO_MPI_master', master)


Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [26]:
X_BO_initRS_LCB_collection = []
run_BO_initRS_LCB_collection = []
loss_BO_initRS_LCB_collection = []

for s in seed_list:
    random.seed(s)
    
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_LCB = []
    run_BO_initRS_LCB = []
    loss_BO_initRS_LCB = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_loss = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(initial_X[j])
                run_BO_initRS_LCB.append(i)
                loss_BO_initRS_LCB.append(initial_loss[j])

            min_round = np.min(initial_loss)
            min_index = np.where(initial_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'LCB',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_LCB),
                                            Y = np.array([[i] for i in loss_BO_initRS_LCB]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

#             suggested_next_loss = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            suggested_next_loss = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(suggested_next_X[j])
                run_BO_initRS_LCB.append(i)
                loss_BO_initRS_LCB.append(suggested_next_loss[j])

            min_round = np.min(suggested_next_loss)
            min_index = np.where(suggested_next_loss == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], loss_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(loss_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_LCB_collection.append(X_BO_initRS_LCB)
    run_BO_initRS_LCB_collection.append(run_BO_initRS_LCB)
    loss_BO_initRS_LCB_collection.append(loss_BO_initRS_LCB)
    print('Finished e')
    
master = np.array([X_BO_initRS_LCB_collection, run_BO_initRS_LCB_collection, loss_BO_initRS_LCB_collection])
np.save('XGB_BO_LCB_master', master)

The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ign